# Ingestion Test - Streaming Availability

In [1]:
from bmb.test.test_util import _temp_copy, PATH_TO_TEST_DB
from bmb.source.FilmDB  import FilmDB
import pandas as pd
import sqlite3

## Create Temporary Test Database
Like all tests, the Steaming Availability Ingestion Test uses a temporary test database (TTD) to avoid interference between tests.

In [2]:
temporary_test_database = _temp_copy(PATH_TO_TEST_DB, unique=True)
db = FilmDB( temporary_test_database)

The `show_log` function displays the webscraping history for this TTD and will be called before and after the streaming availability update operations to demonstrate changes to the `JustWatchLog` table.

In [3]:
def show_log():
    conn = sqlite3.connect(temporary_test_database)
    df = pd.read_sql_query("SELECT * FROM JustWatchLog", conn)
    conn.close()
    return df
show_log()

,streamer,start_year,end_year,genre,total,new,completed


## Streaming Availability Update Operations
`FilmDB.update_streaming_content` records the raw JustWatchURLs, but does not identify the films to which they point, necessitating the final `FilmDB.identify_new_JustWatchURLs` call.

In [4]:
db.update_streaming_content( 'hulu'   , ymax=2000, genre='horror' )
db.update_streaming_content( 'netflix', ymin=1999, ymax=1999)
db.update_streaming_content( 'amazon' , ymin=1980, ymax=2000, genre='horror' )
db.update_streaming_content( 'amazon' , ymin=1980, ymax=2000, genre='science fiction' )
db.identify_new_JustWatchURLs( VERBOSE='jupyter')

  0%|          | 0/174 [00:00<?, ?it/s]

Calling the `show_log` function a second time provides examples of the data recorded for each webscrape.

In [5]:
show_log()

,streamer,start_year,end_year,genre,total,new,completed
0,3,NaN,2000,horror,21,21,2021-10-23 13:49
1,1,1999.0,1999,None,15,15,2021-10-23 13:49
2,2,1980.0,2000,horror,93,93,2021-10-23 13:50
3,2,1980.0,2000,science fiction,65,48,2021-10-23 13:50


## DestroyTemporary Test Database

In [6]:
temporary_test_database.unlink()